# Visit the Wikipedia hyperlinks graph!

The goal of this project is to perform an analysis of the Wikipedia Hyperlink graph. 

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from collections import OrderedDict
from tqdm import tqdm
from functions import *
import json

First of all we import the **wiki-topcats-reduced.txt** file where each row represents an edge (departure node and arrival one).

In [2]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep = '\t', header = None)
df.columns = ['from', 'to']
names = name_creation()
names = pd.DataFrame.from_dict(names, orient='index')
df.head()

,from,to
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [3]:
# directed cause these two are different
print(df[df['from'] ==52], '\n')
print(df[df['from'] ==401135].head())

   from       to
0    52   401135
1    52  1069112
2    52  1163551 

          from      to
361754  401135   60219
361755  401135  167532
361756  401135  400980
361757  401135  401018
361758  401135  401019


By taking a look at the dataframe we notice that the graph is **directed**: in fact the edges are directed from one vertex to another. As we can see in the dataframe, a node could have edges departing from it but no edges to it. In the example presented above, we've seen that the node $52$ goes to the node $401135$ but the inverse edge is not present.

Since the graph is directed we cannot consider only one column of the dataframe thus the number of nodes is given by the length of the *set* of both columns while the number of edges is simply the length of the dataframe.

In [4]:
#vertices

fr = df.groupby('from').count()
to = df.groupby('to').count()
V = set(list(fr.index) + list(to.index))
print('Number of nodes =', len(V))
print('Number of edges:', len(df.index))


Number of nodes = 461193
Number of edges: 2645247


In [5]:
print('The average degree of the vertex is:', round(len(df.index)/len(V), 4))

The average degree of the vertex is: 5.7357


We have found that the average degree of all the vertexes is $5.74$ and this means that the graph is sparse. 
The average degree has been found with the formula:

$average\_degree$ = $\frac{total\_edges}{total\_vertex}$

Now we can work on the **wikipedia categories** and drop those with a number of articles less than $3500$ and create two dictionaries that have the same structure:
* categories = {category_name : nodes}

In [6]:
categories = category_creation(V)

The **categories** dictionary contains the nodes that are present in the initial dataframe

Now it is asked to the user to insert an initial category.

In [8]:
input_category = input()
'Year_of_birth_unknown'

Year_of_birth_unknown


In [9]:
DG = nx.from_pandas_edgelist(df, 'from', 'to', create_using = nx.DiGraph )

In order to find all the shortest paths from the input category to all the others, we have used the **BFS** algorithm. 
We have runned this algorithm for each node present in the initial category. At the end we have a dictionary that has as keys the nodes that have been visited with all the associated shortest paths.
An example of the final dictionary is shown here:

$\{node_i: [dist_1, dist_2,..., dist_n ]\}$ , where n is the number of nodes in the input category.

In [11]:
bfs = BFS(DG, categories, input_category)

100%|████████████████████████████████████████████████████████████████████████████| 2536/2536 [1:16:34<00:00,  1.81s/it]


All the process described above takes a long time, so, in order to save time, we have saved the data that can be accessed faster.

In [10]:
with open('data.json', 'r') as fp:
    data = json.load(fp)

In [11]:
median = median_calculation(categories,input_category, data)

In [39]:
block_ranking = OrderedDict()
block_ranking = OrderedDict(sorted(median.items(), key=lambda x: x[1]))
block_ranking

OrderedDict([('Year_of_birth_unknown', -1),
             ('English-language_films', 6.0),
             ('American_film_actors', 6.0),
             ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
              7.0),
             ('Indian_films', 7.0),
             ('English_television_actors', 7.0),
             ('British_films', 7.0),
             ('American_films', 7.0),
             ('American_Jews', 7.0),
             ('American_television_actors', 7.0),
             ('Black-and-white_films', 7.0),
             ('Article_Feedback_Pilot', 7.0),
             ('Rivers_of_Romania', 8.0),
             ('English-language_albums', 8.0),
             ('People_from_New_York_City', 8.0),
             ('Debut_albums', 9.0),
             ('English_footballers', 10000),
             ('The_Football_League_players', 10000),
             ('Association_football_forwards', 10000),
             ('Association_football_goalkeepers', 10000),
             ('Association_football_mid

The **articles** dictionary contains the articles and the corresponding categories in which it appears:
* articles = {article : list_of_categories}

In [32]:
# map each article to the categories it is present

articles = {}
for name in categories:
    for article in categories[name]:
        if article in articles:
            articles[article].append(name)
        else:
            articles[article] = [name]

It can happen that an article belongs to several categories: in this case we assign it to the input category if the input category is one of these otherwise we choose the category according to block_ranking, i.e. it will belong to the category closest to the input one.

In [33]:
# if an article belongs to multiple categories, choose one according to block_ranking

for article in articles:
    if len(articles[article]) > 1:
        minimum = ''
        for cat in articles[article]:
            if (minimum == '') or (block_ranking[minimum] > block_ranking[cat]):
                minimum = cat
        articles[article] = [minimum]

The dictionary **categories_after_ranking** has the category as key and a list of articles belonging to it.

In [34]:
categories_after_ranking = {}

for i in articles:  # article??
    if articles[i][0] in categories_after_ranking:
        categories_after_ranking[articles[i][0]] +=  [i]
    else:
        categories_after_ranking[articles[i][0]] =  [i]

Given a rank to all the categories, now it is time to assign a rank to all the nodes. This is possible working with subgraphs and weights of the edges.

We have followed these three steps:

*  Starting from the input category, C\0 we have computed its subgraph. Then the weights of all the edges departing from a node of the input category have been updated with the corresponding weight of that node.
* After this, we have computed the subgraph induced by the second category, according to the block ranking, that gives an initial weight to all the nodes in that category. All the weights are updated with those that belong to the in-edges that come from the input category.
* Now the step 2 is repeated for all the categories.

In [35]:
weight_dict = subgraph_calculation(DG, block_ranking, categories_after_ranking)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:29<00:00,  1.34it/s]


In [36]:
#rank per ogni nodo (come nello step3 dell'homework)
rank = []
for i in weight_dict:
    rank += list(OrderedDict(sorted(weight_dict[i].items(), key = lambda x: x[1], reverse = True)).keys())

ranks = {}
for i in range(50):
    ranks[rank[i]] = names.loc[rank[i]].values[0]

In [37]:
pd.DataFrame.from_dict(ranks, orient='index', columns=['Article_name']).head(10)

,Article_name
62684,Diogenes Lartius
170163,Pausanias (geographer)
1656777,L Bu
1656780,Dong Zhuo
169696,Theocritus
1656794,Yuan Shu
170578,Stobaeus
1342864,Penda of Mercia
1343014,Symeon of Durham
1656778,Zhang Fei
